In [2]:
import pandas as pd
import numpy as np

# Load the dataset
df=pd.read_csv("DSBDALExam DataSets/DSBDALExam DataSets/DSBDALExam DataSets/Adult/adult.csv");

df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Replace '?' with NaN
df.replace('?', np.nan, inplace=True)

# Drop rows with any missing values
df_cleaned = df.dropna()

# Remove negative numeric values (not expected in this dataset but for safety)
numeric_cols = df_cleaned.select_dtypes(include=[np.number]).columns
df_cleaned = df_cleaned[(df_cleaned[numeric_cols] >= 0).all(axis=1)]


C:\Users\UTKARSH BRAHMANKAR\AppData\Local\Temp\ipykernel_29324\3732865498.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [4]:
from scipy import stats
import numpy as np

# Calculate Z-scores
z_scores = np.abs(stats.zscore(df_cleaned[numeric_cols]))

# Keep only rows where all Z-scores are below a threshold (e.g., 3)
df_no_outliers = df_cleaned[(z_scores < 3).all(axis=1)]


In [5]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Encode categorical columns
df_transformed = df_no_outliers.copy()
label_encoders = {}

for col in df_transformed.select_dtypes(include='object').columns:
    le = LabelEncoder()
    df_transformed[col] = le.fit_transform(df_transformed[col])
    label_encoders[col] = le

# Separate features and target
X = df_transformed.drop('income', axis=1)
y = df_transformed['income']

# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Logistic Regression
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)
lr_preds = lr_model.predict(X_test)
lr_acc = accuracy_score(y_test, lr_preds)

# Naïve Bayes
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
nb_preds = nb_model.predict(X_test)
nb_acc = accuracy_score(y_test, nb_preds)

# Compare Accuracies
print("Logistic Regression Accuracy:", lr_acc)
print("Naïve Bayes Accuracy:", nb_acc)


Logistic Regression Accuracy: 0.8208720860007247
Naïve Bayes Accuracy: 0.8011837178403188


In [7]:
''' # Use IQR method for numerical outlier removal
Q1 = df_cleaned[numeric_cols].quantile(0.25)
Q3 = df_cleaned[numeric_cols].quantile(0.75)
IQR = Q3 - Q1

# Remove outliers
df_no_outliers = df_cleaned[~((df_cleaned[numeric_cols] < (Q1 - 1.5 * IQR)) |
                              (df_cleaned[numeric_cols] > (Q3 + 1.5 * IQR))).any(axis=1)]
'''

' # Use IQR method for numerical outlier removal\nQ1 = df_cleaned[numeric_cols].quantile(0.25)\nQ3 = df_cleaned[numeric_cols].quantile(0.75)\nIQR = Q3 - Q1\n\n# Remove outliers\ndf_no_outliers = df_cleaned[~((df_cleaned[numeric_cols] < (Q1 - 1.5 * IQR)) |\n                              (df_cleaned[numeric_cols] > (Q3 + 1.5 * IQR))).any(axis=1)]\n'